In [216]:
import pandas as pd
import numpy as np

# Shopify: Fall 2022 Data Science Intern Challenge, Daniel Ho

## Question 1

**Part A:** To begin, I read the csv file and added an extra column to the dataframe (the unit cost of one shoe for any user purchase).

In [217]:
df = pd.read_csv('2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv')
df['shoe_unit_cost'] = df['order_amount'] / df['total_items']
df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv'

Since the mean was so high, I wanted to examine other statistics within the dataset. Some particular things that stood out were the particularly high maximums for order_amount and total_items (704000, 2000 respectively). 

In [218]:
df.describe()

,order_id,shop_id,user_id,order_amount,total_items,shoe_unit_cost
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720,387.742800
std,1443.520003,29.006118,87.798982,41282.539349,116.32032,2441.963725
min,1.000000,1.000000,607.000000,90.000000,1.00000,90.000000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000,133.000000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000,153.000000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000,169.000000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000,25725.000000


To delve further into the orders of each unique user, I grouped by user_id and aggregated by mean, then sorted values by order amount in increasing order. Immediately, we can see that user 607 is a key outlier, and is part of the reason why the AOV is so high. We can also observe that shoe_unit_costs are unusually high towards the end, so additional analysis is needed.

In [201]:
avgs_by_user = df.groupby('user_id').mean()
avgs_by_user.sort_values('order_amount')

,order_id,shop_id,order_amount,total_items,shoe_unit_cost
user_id,,,,,
864,2551.210526,36.894737,209.157895,1.473684,148.789474
939,3561.700000,42.100000,219.600000,1.500000,145.300000
827,2573.571429,49.357143,226.500000,1.500000,155.500000
899,3228.066667,56.933333,226.666667,1.666667,134.933333
892,2432.461538,47.846154,227.307692,1.538462,148.076923
...,...,...,...,...,...
915,2519.571429,68.142857,5785.142857,2.071429,1973.785714
834,2655.888889,56.555556,6019.000000,2.111111,1580.833333
766,2391.800000,52.800000,8007.600000,2.300000,2704.900000


Following what I saw in the previous step, I decided to examine the user averages sorted by shoe_unit_cost. Based on the numbers below and the percentile numbers from df.describe earlier, we can deduce that some user averages have high shoe_unit_costs because of large individual purchases.

In [202]:
avgs_by_user.sort_values('shoe_unit_cost')

,order_id,shop_id,order_amount,total_items,shoe_unit_cost
user_id,,,,,
899,3228.066667,56.933333,226.666667,1.666667,134.933333
973,2990.818182,47.363636,278.363636,2.181818,137.636364
806,1134.700000,41.900000,257.200000,1.900000,138.300000
826,3042.500000,38.166667,299.833333,2.166667,138.333333
964,2356.153846,43.769231,332.846154,2.461538,138.384615
...,...,...,...,...,...
867,1856.636364,50.363636,2619.090909,2.090909,2468.545455
878,2543.454545,60.909091,14266.909091,2.181818,2468.818182
766,2391.800000,52.800000,8007.600000,2.300000,2704.900000


Through looking the data, I decided a better way to evaluate AOV was by filtering out the outliers seen above. This includes filtering out user 607, as well as orders in which shoes were particularly expensive. 

In [203]:
new_df = df[(df.user_id != 607) & (df.shoe_unit_cost < 1000)]
new_df.describe()

,order_id,shop_id,user_id,order_amount,total_items,shoe_unit_cost
count,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000
mean,2499.551347,49.846465,849.752279,302.580514,1.994734,151.788536
std,1444.069407,29.061131,86.840313,160.804912,0.982821,29.034215
min,1.000000,1.000000,700.000000,90.000000,1.000000,90.000000
25%,1248.000000,24.000000,775.000000,163.000000,1.000000,132.000000
50%,2497.000000,50.000000,850.000000,284.000000,2.000000,153.000000
75%,3751.000000,74.000000,925.000000,387.000000,3.000000,166.000000
max,5000.000000,100.000000,999.000000,1760.000000,8.000000,352.000000


**Part B/C:** Without actually filtering the original data, a metric I would choose for the dataset is the **50th percentile of the order_amount**. In the original dataframe, the **value of our metric would be 284**, which is actually quite close to the AOV of the new filtered dataframe above (302). It would serve as a good metric because it would account for huge outliers like user 607 and provide a more realistic estimate of AOV. 

## Question 2

**Part A**

SELECT COUNT(*) 
<br />
FROM Orders AS o JOIN Shippers AS s
<br />
ON o.ShipperID = s.ShipperID
<br />
WHERE o.ShipperID = 1;

54

**Part B**

SELECT e.LastName
<br />
FROM Orders AS o JOIN Employees AS e
<br />
ON o.EmployeeID = e.EmployeeID
<br />
GROUP BY e.EmployeeID 
<br />
ORDER BY COUNT(*) DESC
<br />
LIMIT 1;

Dodsworth

**Part C**

SELECT ProductName
<br />
FROM OrderDetails AS od JOIN Orders AS o
<br />
on od.OrderID = o.OrderID 
<br />
JOIN Customers AS c
<br />
ON c.CustomerID = o.CustomerID
<br />
JOIN Products AS p
<br />
ON od.ProductID = p.ProductID
<br />
WHERE c.Country = 'Germany'
<br />
GROUP BY od.ProductID 
<br />
ORDER BY SUM(od.Quantity)
<br />
LIMIT 1;

Boston Crab Meat